In [6]:
import pandas as pd
import re

import _codes
import _util
import _config

#re
pattern = re.compile(r'\d/4')

#file_path setting
files = _util.get_files(_config.hospital_read_path)

df_list = []
for file in files:
    df_list.append(pd.read_csv(_config.hospital_read_path + '\\' + file, encoding=_config.euc_kr))
df = pd.concat(df_list, ignore_index=False)

_cols = {}
for col in df.columns:
    if col.count('20') > 0 and col.count('.1/4') > 0: _cols[col] = str(col).replace('.1/4', '')
    elif col.count('Unnamed') > 0: None
    elif not pattern.search(col): _cols[col] = col

df.rename(columns=_cols, inplace=True)
df = df[list(_cols.values())]
df = pd.DataFrame(df)

print(df.columns)
# df.to_csv(_config.hospital_write_path + '\\' + 'hospital' + '.csv', encoding=_config.euc_kr, index=False)

num_cols = [col for col in df.columns if col.isnumeric()]
not_num_cols = [col for col in df.columns if not col.isnumeric()]

df = df.melt(
    id_vars=not_num_cols
    , value_vars=num_cols
    , var_name='year'
    , value_name='개수'
    , ignore_index=False
)
df.reset_index(drop=True, inplace=True)

sido_cd = None
for idx, row in df.iterrows():
    sgg = row['시군구별']

    #year
    year = str(row['year'])
    year = int(year.strip())

    case_no = 2
    sido_sgg_cd_map = None
    if 2005 < year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2000, case_no)
    elif 2010 < year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2005, case_no)
    elif 2015 < year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2010, case_no)
    elif 2020 < year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2015, case_no)
    elif 2023 < year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2020, case_no)
    elif 2023 >= year: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2023, case_no) 

    #sido_cd mapping
    sido_cd = _util.sido_cd_map.get(sgg, sido_cd)

    #sgg_cd mapping2
    sido = sido_sgg_cd_map.get(sido_cd, {})
    sido = dict(sido)
    sgg_cd = sido.get(sgg, None)

    #예외
    if sgg == '전라북도': sgg_cd = '35'

    df.loc[idx, 'sgg_cd'] = f'`{str(sgg_cd)}'
    df.loc[idx, 'sido_cd'] = f'`{sido_cd}'

df = df.pivot_table(
    index=['sido_cd', 'sgg_cd', '시군구별', 'year']
    , columns='요양기관종별'
    , values='개수'
    , aggfunc='sum'
)
df.reset_index(inplace=True)
df.to_csv(_config.hospital_write_path + '\\' + 'hospital' + '.csv', encoding=_config.euc_kr, index=False)
print("end write file")


Index(['시군구별', '요양기관종별', '항목', '단위', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023'],
      dtype='object')


In [11]:
def create_table(columns):
    columns = list(columns)
    for idx in range(len(columns)):
        # if str(columns[idx]).count('year') > 0: 
        columns[idx] = f'"{columns[idx]}"'

        print(columns[idx])
        if str(columns[idx]).count('sido_cd') > 0: columns[idx] = columns[idx] + ' varchar(2)'
        elif str(columns[idx]).count('sgg_cd') > 0: columns[idx] = columns[idx] + ' varchar(6)'
        elif str(columns[idx]).count('시군구별') > 0: columns[idx] = columns[idx] + ' varchar(20)'
        elif str(columns[idx]).count('year') > 0: columns[idx] = columns[idx] + ' varchar(4)'
        else: columns[idx] = columns[idx] + ' float8'

    sql = f'''create table if not exists hospital(
        {', '.join(columns)}
    )
    '''
    cursor.execute(sql)
    conn.commit()
    cursor.close()
    conn.close()

    #insert
def data_insert(df, columns):
    conn = psycopg2.connect('dbname=postgres user=postgres password=force1234 host=localhost port=5432')
    cursor = conn.cursor()

    #column setting
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

    #to use dataframe for clause
    for idx, row in df.iterrows():
        row = row.tolist()
            
        for idx in range(len(columns)):
            if str(row[idx]).count('`') > 0: row[idx] = str(row[idx]).replace('`', '')
            row[idx] = f'\'{str(row[idx])}\''

            if row[idx] == 'None': row[idx] = '\'\''

        sql = f'''
            INSERT INTO hospital(
                {','.join(columns)})
            VALUES(
                {', '.join(row)}
            )
        '''
        print(sql)
        cursor.execute(sql)
        # try:
        #     cursor.execute(sql)
        # except Exception:
        #     print('sql 실행도중 에러 발생')

    conn.commit()
    cursor.close()
    conn.close()

file_name = 'hospital' + '.csv'
file_path = input_dir_path + '\\' + file_name

df = pd.read_csv(file_path, encoding=euc_kr)
create_table(df.columns)
data_insert(df, df.columns)



"sido_cd"
"sgg_cd"
"시군구별"
"year"
"계"
"병원"
"보건소"
"보건의료원"
"보건지소"
"보건진료소"
"상급종합병원"
"약국"
"요양병원"
"의원"
"정신병원"
"조산원"
"종합병원"
"치과병원"
"치과의원"
"한방병원"
"한의원"

            INSERT INTO hospital(
                "sido_cd","sgg_cd","시군구별","year","계","병원","보건소","보건의료원","보건지소","보건진료소","상급종합병원","약국","요양병원","의원","정신병원","조산원","종합병원","치과병원","치과의원","한방병원","한의원")
            VALUES(
                '00', '00', '전체', '2009', '78829.0', '1211.0', '239.0', '17.0', '1286.0', '1912.0', '44.0', '20880.0', '714.0', '26640.0', '0.0', '51.0', '269.0', '172.0', '13831.0', '139.0', '11424.0'
            )
        

            INSERT INTO hospital(
                "sido_cd","sgg_cd","시군구별","year","계","병원","보건소","보건의료원","보건지소","보건진료소","상급종합병원","약국","요양병원","의원","정신병원","조산원","종합병원","치과병원","치과의원","한방병원","한의원")
            VALUES(
                '00', '00', '전체', '2010', '80632.0', '1284.0', '239.0', '17.0', '1291.0', '1915.0', '44.0', '21036.0', '800.0', '27167.0', '0.0', '46.0', '271.0', '185.0', '14321.0', '165.0', '11851.